In [19]:
import duckdb

# DuckDB 연결 설정 (인메모리 데이터베이스)
con = duckdb.connect(database=':memory:', read_only=False)

# ----------------------------------------------------
# '직원 정보 (Employees)' 테이블 생성
# 조인 키 이름: 부서코드
# ----------------------------------------------------
print("1. 'Employees' 테이블 생성 및 데이터 삽입")
con.execute("""
CREATE TABLE Employees (
    직원ID INTEGER PRIMARY KEY,
    이름 VARCHAR(50),
    부서코드 VARCHAR(10),
    입사일 DATE
);
""")

con.execute("""
INSERT INTO Employees (직원ID, 이름, 부서코드, 입사일) VALUES
(101, '김민수', 'D01', '2020-01-15'),
(102, '박서연', 'D02', '2021-03-20'),
(103, '이준호', 'D01', '2019-11-01'),
(104, '정하은', 'D03', '2022-07-10'),
(105, '최지훈', 'D04', '2023-05-25');
""")

print("\n--- [테이블 1] Employees 데이터 확인 ---")
print(con.execute("SELECT * FROM Employees ORDER BY 직원ID;").fetchdf())


1. 'Employees' 테이블 생성 및 데이터 삽입

--- [테이블 1] Employees 데이터 확인 ---
   직원ID   이름 부서코드        입사일
0   101  김민수  D01 2020-01-15
1   102  박서연  D02 2021-03-20
2   103  이준호  D01 2019-11-01
3   104  정하은  D03 2022-07-10
4   105  최지훈  D04 2023-05-25


In [20]:

# ----------------------------------------------------
# '부서 정보 (Departments)' 테이블 생성
# 조인 키 이름: 코드
# ----------------------------------------------------
print("2. 'Departments' 테이블 생성 및 데이터 삽입")
con.execute("""
CREATE TABLE Departments (
    코드 VARCHAR(10) PRIMARY KEY,
    부서명 VARCHAR(50),
    위치 VARCHAR(50)
);
""")

con.execute("""
INSERT INTO Departments (코드, 부서명, 위치) VALUES
('D01', '영업팀', '본사'),
('D04', '인사팀', '별관'),
('D02', '마케팅팀', '지점'),
('D05', '연구소', '연구동');
""")

print("\n--- [테이블 2] Departments 데이터 확인 ---")
print(con.execute("SELECT * FROM Departments ORDER BY 코드;").fetchdf())


2. 'Departments' 테이블 생성 및 데이터 삽입

--- [테이블 2] Departments 데이터 확인 ---
    코드   부서명   위치
0  D01   영업팀   본사
1  D02  마케팅팀   지점
2  D04   인사팀   별관
3  D05   연구소  연구동


In [21]:

# ----------------------------------------------------
# INNER JOIN 쿼리 실행
# 두 테이블의 키 이름이 다른 상황: E.부서코드 = D.코드
# ----------------------------------------------------
inner_join_query = """
SELECT
    E.직원ID,
    E.이름,
    D.부서명,
    D.위치,
    E.입사일
FROM Employees AS E
INNER JOIN Departments AS D
  ON E.부서코드 = D.코드
ORDER BY E.직원ID;
"""

print("\n--- 3. INNER JOIN 결과 (E.부서코드 = D.코드) ---")
result_df = con.execute(inner_join_query).fetchdf()
print(result_df)


con.close()


--- 3. INNER JOIN 결과 (E.부서코드 = D.코드) ---
   직원ID   이름   부서명  위치        입사일
0   101  김민수   영업팀  본사 2020-01-15
1   102  박서연  마케팅팀  지점 2021-03-20
2   103  이준호   영업팀  본사 2019-11-01
3   105  최지훈   인사팀  별관 2023-05-25


In [22]:
#       직원ID   이름   부서코드          입사일                  코드   부서명   위치
# 0     101     김민수  D01             2020-01-15          0  D01   영업팀    본사
# 1     102     박서연  D02             2021-03-20          1  D02   마케팅팀   지점
# 2     103     이준호  D01             2019-11-01          2  D04   인사팀    별관
# 3     104     정하은  D03             2022-07-10          3  D05   연구소    연구동
# 4     105     최지훈  D04             2023-05-25

In [23]:
import duckdb

# 인메모리 데이터베이스 연결
con = duckdb.connect(database=':memory:', read_only=False)

# ----------------------------------------------------
# '주문 정보 (Orders)' 테이블 생성 및 데이터 삽입
# 조인 키 이름: 고객번호
# ----------------------------------------------------
print("1. 'Orders' 테이블 생성 및 데이터 삽입")
con.execute("""
CREATE TABLE Orders (
    주문ID INTEGER PRIMARY KEY,
    고객번호 INTEGER,  -- Orders 테이블의 조인 키 (이름 다름)
    주문금액 INTEGER,
    주문일 DATE
);
""")

con.execute("""
INSERT INTO Orders (주문ID, 고객번호, 주문금액, 주문일) VALUES
(1001, 10, 55000, '2024-10-01'),
(1002, 30, 120000, '2024-10-05'),
(1003, 20, 30000, '2024-10-07'),
(1004, 10, 88000, '2024-10-10'),
(1005, 40, 45000, '2024-10-12');
""")

print("\n--- [테이블 1] Orders 데이터 확인 (조인 키: 고객번호) ---")
print(con.execute("SELECT * FROM Orders ORDER BY 주문ID;").fetchdf())


1. 'Orders' 테이블 생성 및 데이터 삽입

--- [테이블 1] Orders 데이터 확인 (조인 키: 고객번호) ---
   주문ID  고객번호    주문금액        주문일
0  1001    10   55000 2024-10-01
1  1002    30  120000 2024-10-05
2  1003    20   30000 2024-10-07
3  1004    10   88000 2024-10-10
4  1005    40   45000 2024-10-12


In [24]:

# ----------------------------------------------------
# '고객 정보 (Customers)' 테이블 생성 및 데이터 삽입
# 조인 키 이름: ID
# ----------------------------------------------------
print("2. 'Customers' 테이블 생성 및 데이터 삽입")
con.execute("""
CREATE TABLE Customers (
    ID INTEGER PRIMARY KEY, -- Customers 테이블의 조인 키 (이름 다름)
    이름 VARCHAR(50),
    도시 VARCHAR(50)
);
""")

con.execute("""
INSERT INTO Customers (ID, 이름, 도시) VALUES
(10, '김철수', '서울'),
(40, '박민지', '부산'),
(20, '이영희', '인천'),
(50, '최지영', '대구');
""")

print("\n--- [테이블 2] Customers 데이터 확인 (조인 키: ID) ---")
print(con.execute("SELECT * FROM Customers ORDER BY ID;").fetchdf())


2. 'Customers' 테이블 생성 및 데이터 삽입

--- [테이블 2] Customers 데이터 확인 (조인 키: ID) ---
   ID   이름  도시
0  10  김철수  서울
1  20  이영희  인천
2  40  박민지  부산
3  50  최지영  대구


In [25]:

# ----------------------------------------------------
# INNER JOIN 쿼리 실행
# 두 테이블의 키 이름이 다름: O.고객번호 = C.ID
# ----------------------------------------------------
inner_join_query = """
SELECT
    O.주문ID,
    C.이름 AS 고객이름,
    C.도시,
    O.주문금액,
    O.주문일
FROM Orders AS O
INNER JOIN Customers AS C
  ON O.고객번호 = C.ID  -- **키 이름이 서로 다름**
ORDER BY O.주문ID;
"""

print("\n--- 5. INNER JOIN 결과 (O.고객번호 = C.ID) ---")
result_df = con.execute(inner_join_query).fetchdf()
print(result_df)


# 연결 닫기
con.close()


--- 5. INNER JOIN 결과 (O.고객번호 = C.ID) ---
   주문ID 고객이름  도시   주문금액        주문일
0  1001  김철수  서울  55000 2024-10-01
1  1003  이영희  인천  30000 2024-10-07
2  1004  김철수  서울  88000 2024-10-10
3  1005  박민지  부산  45000 2024-10-12


In [26]:
#       주문ID  고객번호      주문금액         주문일               ID   이름    도시
# 0     1001    10          55000           2024-10-01      0   10   김철수  서울
# 1     1002    30          120000          2024-10-05      1   20   이영희  인천
# 2     1003    20          30000           2024-10-07      2   40   박민지  부산
# 3     1004    10          88000           2024-10-10      3   50   최지영  대구
# 4     1005    40          45000           2024-10-12      

In [27]:
# Left Join

In [28]:
import duckdb

# 인메모리 데이터베이스 연결
con = duckdb.connect(database=':memory:', read_only=False)

# ----------------------------------------------------
# '주문 정보 (Orders)' 테이블 생성 및 데이터 삽입 (왼쪽 테이블)
# 조인 키 이름: 고객번호
# ----------------------------------------------------
print("1. 'Orders' 테이블 생성 및 데이터 삽입")
con.execute("""
CREATE TABLE Orders (
    주문ID INTEGER PRIMARY KEY,
    고객번호 INTEGER,  -- 조인 키 (이름 다름)
    주문금액 INTEGER,
    주문일 DATE
);
""")

con.execute("""
INSERT INTO Orders (주문ID, 고객번호, 주문금액, 주문일) VALUES
(1001, 10, 55000, '2024-10-01'),
(1002, 30, 120000, '2024-10-05'),
(1003, 20, 30000, '2024-10-07'),
(1004, 10, 88000, '2024-10-10'),
(1005, 40, 45000, '2024-10-12');
""")

print("\n--- [테이블 1] Orders 데이터 확인 (LEFT JOIN 기준) ---")
print(con.execute("SELECT * FROM Orders ORDER BY 주문ID;").fetchdf())


1. 'Orders' 테이블 생성 및 데이터 삽입

--- [테이블 1] Orders 데이터 확인 (LEFT JOIN 기준) ---
   주문ID  고객번호    주문금액        주문일
0  1001    10   55000 2024-10-01
1  1002    30  120000 2024-10-05
2  1003    20   30000 2024-10-07
3  1004    10   88000 2024-10-10
4  1005    40   45000 2024-10-12


In [29]:
# ----------------------------------------------------
# '고객 정보 (Customers)' 테이블 생성 및 데이터 삽입 (오른쪽 테이블)
# 조인 키 이름: ID
# ----------------------------------------------------
print("2. 'Customers' 테이블 생성 및 데이터 삽입")
con.execute("""
CREATE TABLE Customers (
    ID INTEGER PRIMARY KEY, -- 조인 키 (이름 다름)
    이름 VARCHAR(50),
    도시 VARCHAR(50)
);
""")

con.execute("""
INSERT INTO Customers (ID, 이름, 도시) VALUES
(10, '김철수', '서울'),
(40, '박민지', '부산'),
(20, '이영희', '인천'),
(50, '최지영', '대구');
""")

print("\n--- [테이블 2] Customers 데이터 확인 ---")
print(con.execute("SELECT * FROM Customers ORDER BY ID;").fetchdf())


2. 'Customers' 테이블 생성 및 데이터 삽입

--- [테이블 2] Customers 데이터 확인 ---
   ID   이름  도시
0  10  김철수  서울
1  20  이영희  인천
2  40  박민지  부산
3  50  최지영  대구


In [ ]:
# Left Join

# LEFT JOIN은 두 테이블을 결합할 때, 왼쪽(기준) 테이블의 모든 행을 유지하고 오른쪽 테이블에서 일치하는 행을 찾아 결합하는 방식으로 동작

# Orders (왼쪽)	                            Customers (오른쪽)
# 주문ID, 고객번호, 주문금액, 주문일	        ID, 이름, 도시
# 1001,  10,      55000,   2024-10-01	    10, 김철수, 서울
# 1002,  30,      120000,  2024-10-05	    40, 박민지, 부산
# 1003,  20,      30000,   2024-10-07	    20, 이영희, 인천
# 1004,  10,      88000,   2024-10-10	    50, 최지영, 대구
# 1005,  40,      45000,   2024-10-12

In [ ]:
# 주문ID	고객번호	주문금액	주문일             |        고객이름	도시
# 1001	    10	       55000	 2024-10-01        |        김철수	    서울
# 1002	    30	       120000	 2024-10-05        |        None	   None
# 1003	    20	       30000	 2024-10-07        |        이영희	    인천
# 1004	    10	       88000	 2024-10-10        |        김철수	    서울
# 1005	    40	       45000	 2024-10-12        |        박민지	    부산

In [31]:

# ----------------------------------------------------
# LEFT JOIN 쿼리 실행
# 왼쪽 테이블(Orders)의 모든 행을 유지합니다.
# ----------------------------------------------------
left_join_query = """
SELECT
    O.주문ID,
    O.고객번호,
    C.이름 AS 고객이름,
    C.도시,
    O.주문금액,
    O.주문일
FROM Orders AS O
LEFT JOIN Customers AS C
  ON O.고객번호 = C.ID
ORDER BY O.주문ID;
"""

print("\n--- LEFT JOIN 결과 (Orders 기준) ---")
result_df = con.execute(left_join_query).fetchdf()
print(result_df)

# ----------------------------------------------------
# 연결 닫기
con.close()


--- LEFT JOIN 결과 (Orders 기준) ---
   주문ID  고객번호  고객이름    도시    주문금액        주문일
0  1001    10   김철수    서울   55000 2024-10-01
1  1002    30  None  None  120000 2024-10-05
2  1003    20   이영희    인천   30000 2024-10-07
3  1004    10   김철수    서울   88000 2024-10-10
4  1005    40   박민지    부산   45000 2024-10-12


In [32]:
# Left Join - 2

In [33]:
import duckdb
import pandas as pd

# DuckDB 연결 설정 (인메모리 데이터베이스)
con = duckdb.connect(database=':memory:', read_only=False)
print("1. DuckDB 연결 완료.")

# ----------------------------------------------------
# '프로젝트 정보 (Projects)' 테이블 생성 및 데이터 삽입 (왼쪽 테이블)
# ----------------------------------------------------
con.execute("""
CREATE TABLE Projects (
    프로젝트ID INTEGER PRIMARY KEY,
    Project_Code VARCHAR(10),  -- Projects 테이블의 조인 키
    프로젝트명 VARCHAR(50),
    시작일 DATE
);
""")

con.execute("""
INSERT INTO Projects (프로젝트ID, Project_Code, 프로젝트명, 시작일) VALUES
(1, 'P001', '차세대 앱 개발', '2024-01-01'),
(2, 'P002', '보안 시스템 개선', '2024-03-15'),
(3, 'P003', '데이터 플랫폼 구축', '2024-05-20'),
(4, 'P004', 'AI 모델 학습', '2024-06-01');
""")

print("\n--- [테이블 1] Projects 데이터 확인 (LEFT JOIN 기준) ---")
print(con.execute("SELECT * FROM Projects ORDER BY 프로젝트ID;").fetchdf())


1. DuckDB 연결 완료.

--- [테이블 1] Projects 데이터 확인 (LEFT JOIN 기준) ---
   프로젝트ID Project_Code       프로젝트명        시작일
0       1         P001    차세대 앱 개발 2024-01-01
1       2         P002   보안 시스템 개선 2024-03-15
2       3         P003  데이터 플랫폼 구축 2024-05-20
3       4         P004    AI 모델 학습 2024-06-01


In [34]:

# '직원 할당 정보 (Assignments)' 테이블 생성 및 데이터 삽입 (오른쪽 테이블)

con.execute("""
CREATE TABLE Assignments (
    할당ID INTEGER PRIMARY KEY,
    Code VARCHAR(10), -- Assignments 테이블의 조인 키 (이름 다름)
    직원명 VARCHAR(50),
    투입률 INTEGER
);
""")

con.execute("""
INSERT INTO Assignments (할당ID, Code, 직원명, 투입률) VALUES
(101, 'P002', '이철수', 100),
(102, 'P001', '박영희', 50),
(103, 'P005', '김민수', 80),
(104, 'P001', '최지훈', 100);
""")

print("\n--- [테이블 2] Assignments 데이터 확인 ---")
print(con.execute("SELECT * FROM Assignments ORDER BY 할당ID;").fetchdf())



--- [테이블 2] Assignments 데이터 확인 ---
   할당ID  Code  직원명  투입률
0   101  P002  이철수  100
1   102  P001  박영희   50
2   103  P005  김민수   80
3   104  P001  최지훈  100


In [35]:
# LEFT JOIN 쿼리 실행
# P.Project_Code = A.Code (서로 다른 이름의 키 필드를 연결)

left_join_query = """
SELECT
    P.프로젝트ID,
    P.프로젝트명,
    A.직원명,
    A.투입률,
    P.시작일
FROM Projects AS P
LEFT JOIN Assignments AS A
  ON P.Project_Code = A.Code
ORDER BY P.프로젝트ID, A.직원명;
"""

print("\n--- 6. LEFT JOIN 최종 결과 (Projects 기준) ---")
result_df = con.execute(left_join_query).fetchdf()
print(result_df)

# 6. 연결 닫기
con.close()


--- 6. LEFT JOIN 최종 결과 (Projects 기준) ---
   프로젝트ID       프로젝트명   직원명   투입률        시작일
0       1    차세대 앱 개발   박영희    50 2024-01-01
1       1    차세대 앱 개발   최지훈   100 2024-01-01
2       2   보안 시스템 개선   이철수   100 2024-03-15
3       3  데이터 플랫폼 구축  None  <NA> 2024-05-20
4       4    AI 모델 학습  None  <NA> 2024-06-01
